# --- Day 11: Reactor ---


In [1]:
# --- Support Functions ---
# This section contains support functions used by the main code.
!pip install -q tqdm

from tqdm.notebook import tqdm
import time

def read_input(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()
    
def traverse_tree(start: str, end: str, tree: dict) -> list[list]:
    paths=[]
    queue = [[start]]
    pbar = tqdm(desc="Tree Exploration", dynamic_ncols=True)
    idx = 0
    while queue:
        path = queue.pop()
        idx +=1    
        pbar.set_description(f"Visito: {idx}: {path[-1]} | Trovate: {len(paths)}")
        for b in tree.get(path[-1], []):
            p = path.copy()+[b]
            if b == end:
                paths.append(p)
                pbar.update(1)
                pbar.set_description(f"Trovate: {len(paths)}")
                break
            queue.append(p)
    pbar.close()
    return(paths)

def rev_tree(tree: dict) -> dict:
    rtree = {}
    for p, cs in tree.items():
        for c in cs:
            rtree[c] = rtree.get(c, []) + [p]
    return rtree

def can_reach(target : str, tree: dict) -> set:
    cr = set() | {target}
    queue = [target]
    while queue:
        node = queue.pop(0)
        ns = tree.get(node, [])
        if not ns:
            continue
        # print (f"node: {node}, queue = {queue}, cr = {cr}")
        for n in ns:
            if n in cr:
                continue
            cr.add(n)
            queue.append(n)
    return cr

def traverse_pruned(
    start: str, end: str, tree: dict[str, list[str]], 
    can_fft: set[str], can_dac: set[str], can_out: set[str]) -> list[list[str]]:
    results=[]
    stack = [(start, [start], {start}, start == 'fft', start == 'dac')] # node, [path], {seen}

    pbar = tqdm(desc="Tree Exploration", dynamic_ncols=True)
    visited = 0

    while stack:
        node, path, seen, has_fft, has_dac = stack.pop(0)

        visited +=1    
        pbar.set_description(f"Visit: {visited}: {node} | Found: {len(results)}")

        # global pruning
        if node not in can_out:
            continue
        if not has_fft and node not in can_fft:
            continue
        if not has_dac and node not in can_dac:
            continue

        for child in tree.get(node, []):
            if child in seen:
                continue
            n_fft = has_fft or child == 'fft'
            n_dac = has_dac or child == 'dac'
            if child == end:
                if n_fft and n_dac:
                    results.append(path + [child])
                    pbar.update(1)
                    pbar.set_description(f"Trovate: {len(results)}")
                continue
            
            stack.append((child, path, seen | {child}, n_fft, n_dac ))

    pbar.close()
    return results



    

In [2]:
# --- Part ONE ---

input = read_input('input.txt')

start = 'you'
end = 'out'

# construct "tree" of parents and childs
childs = {}
parents = {}  # not necessary
for l in input:
    p, cs = l.split(':')
    cs = cs.strip().split(' ')
    childs[p] = cs
    for c in cs:
        pp = (parents.get(c, []))
        pp.append(p)
        parents[c] = pp
# print(f"{childs}")
# print(f"{parents}")

paths = traverse_tree(start, end, childs)

result = len(paths)

print("Part ONE:", result)

Tree Exploration: 0it [00:00, ?it/s]

Part ONE: 782


In [3]:
# --- Part TWO ---
from functools import cache

input = read_input('input.txt')

result = 0

start = 'svr'
end = 'out'

# construct "tree" of parents and childs
childs = {}
for l in input:
    p, cs = l.split(':')
    cs = cs.strip().split(' ')
    childs[p] = cs

# print(childs)

@cache
def count_paths(node, end, seen_dac = False, seen_fft = False):
    if node == end:
        return 1 if seen_dac and seen_fft else 0
    ret = 0
    for nxt in childs.get(node,[]):
        new_seen_dac = seen_dac or nxt == 'dac'
        new_seen_fft = seen_fft or nxt == 'fft'
        ret += count_paths(nxt, end, new_seen_dac, new_seen_fft)
    return ret

result = count_paths(start, end)

print("Part TWO:", result)

Part TWO: 401398751986160
